In [1]:
!python -c "import sys; print(sys.executable)"

/home/mehboobali/miniconda3/envs/office_env/bin/python


# MLFlow lab

In [2]:
import pandas as pd

In [3]:
pd.__version__

'2.0.0'

### Setting up MLFlow tracking server

We also specify artifact root and backend store URI. This makes it possible to store models.

After running this command tracking server will be accessible at `localhost:5000`

In [4]:
%%bash --bg

mlflow server --host 0.0.0.0 \
    --port 5000 \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./mlruns

### MLProject file

This file is used to configure MLFlow steps.

Using `MLproject` we can define our project's pipeline steps, called *entry points*.

Each entry point in this file corresponds to a shell command.

Entry points can be ran using

```
mlflow run -e <ENTRY_POINT>
```

By default `mlflow run` runs `main` entrypoint.

In [5]:
%cat MLproject

name: basic_mlflow

# this file is used to configure Python package dependencies.
# it uses Anaconda, but it can be also alternatively configured to use pip.
conda_env: conda.yaml

# entry points can be ran using `mlflow run <project_name> -e <entry_point_name>
entry_points:
  download_data:
    # you can run any command using MLFlow
    command: "bash download_data.sh"
  # MLproject file has to have main entry_point. It can be toggled without using -e option.
  main:
    # parameters is a key-value collection.
    parameters:
      file_name:
        type: str
        default: "wine_dataset.csv"
      random_state:
        type: int
        default: 10
    command: "python train.py {file_name} {random_state}"


First we need to download data. We will use weather data from previous machine learning tutorial.

## Training

Now we can train models. See `train.py`.
It contains code from supervised machine learning tutorial; we added tracking metrics and model.

We will train kNN models for $k \in \{1, 2, ..., 10\}$ using *temperature* and *casual* features.

After running this command you can go to `localhost:5000` and see the trained models.

In [6]:
import sklearn

In [7]:
sklearn.__version__

'1.2.2'

In [8]:
%%bash
source mlflow_env_vars.sh
mlflow run . 

2023/05/08 09:50:54 INFO mlflow.utils.conda: Conda environment mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 already exists.
2023/05/08 09:50:54 INFO mlflow.projects.utils: === Created directory /tmp/tmpu83f7aws for downloading remote URIs passed to arguments of type 'path' ===
2023/05/08 09:50:54 INFO mlflow.projects.backend.local: === Running command 'source /home/mehboobali/miniconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 1>&2 && python train.py wine_dataset.csv 10' in run with ID 'dbe4de94554448e698be24be60d94f4e' === 
/home/mehboobali/miniconda3/envs/mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'Logistic_reg' already exists. Creating a new version of this model...
2023/05/08 09:50:57 INFO mlflow.tracking._model_registry.client: Waiting up to 

## Inspecting stored models

The trained models are stored in `mlruns/0`.

These directories contain artifacts and config that is needed to serve them.

In [9]:
%%bash
last_model_path=$(ls -tr mlruns/0/ | tail -1)
cat mlruns/0/$last_model_path/artifacts/lr/MLmodel

artifact_path: lr
flavors:
  python_function:
    env:
      conda: conda.yaml
      virtualenv: python_env.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.11.3
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.2.2
mlflow_version: 2.3.1
model_uuid: 5a62e538a50d41dc81a9c9be410aa846
run_id: 21bbe8de85a0478ca1a27fa6d6d11074
utc_time_created: '2023-05-08 04:51:05.774076'


In [10]:
import mlflow

In [11]:
mlflow.__version__

'2.3.1'

## Serving model

Now that we trained our models we can go to *Models* page on MLFLow UI (http://localhost:5000/#/models).

Click *sklearn_knn* on this page, choose a model and move it to *Production* stage.

The following cell will serve the model at localhost on port 5001.

In [12]:
%%bash --bg
source mlflow_env_vars.sh
mlflow --version
mlflow models serve -m models:/Logistic_reg/Production -p 5001 --env-manager=conda 



# Prediction

We'll load data that we can feed into prediction server.

In [13]:
df = pd.read_csv("wine_dataset.csv")[['alcohol', 'flavanoids', 'color_intensity', 'proline']]
df.head()

,alcohol,flavanoids,color_intensity,proline
0,14.23,3.06,5.64,1065.0
1,13.20,2.76,4.38,1050.0
2,13.16,3.24,5.68,1185.0
3,14.37,3.49,7.80,1480.0
4,13.24,2.69,4.32,735.0


Let's predict for first winter day and first non-winter day (first rows of previous two dataframes)

**warning: this might fail at first because the prediction server didn't spin up; in this case wait a minute**

In [14]:
%%bash
data='[[13.20, 2.56, 9.31, 1375]]'
echo $data

curl -d "{\"inputs\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[13.20, 2.56, 9.31, 1375]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    59  100    20  100    39   3381   6594 --:--:-- --:--:-- --:--:-- 11800


{"predictions": [0]}

In [15]:
%%bash
data='[[13.95, 1.15, 7.99, 1084]]'
echo $data

curl -d "{\"instances\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[13.95, 1.15, 7.99, 1084]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    62  100    20  100    42   7800  16380 --:--:-- --:--:-- --:--:-- 31000


{"predictions": [2]}

In [16]:
%%bash
data='[[11.00, 3.33, 5.66, 1400]]'
columns='["alcohol","flavanoids", "color_intensity", "proline"]'
echo $data

curl -d "{\"dataframe_split\":{\"columns\":[\"alcohol\",\"flavanoids\", \"color_intensity\", \"proline\"],\"data\": $data}}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[11.00, 3.33, 5.66, 1400]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   142  100    20  100   122   3652  22279 --:--:-- --:--:-- --:--:-- 28400


{"predictions": [0]}

Voila! We see that the model outputs correct predictions.